### Get necessary packages

In [72]:
import pandas as pd
import os
import pandasql as ps
import numpy as np
import datetime
from datetime import datetime as dt, timedelta as td

#### Map the companies with transcript with CALS with their transcript availability date to identify the possible 1 for our data 

In [73]:
## Read data with transcript names and their publush date for companies with CALS

data = pd.read_csv('D:\\Chubb\\Company check\\transcript_with_cal.csv', sep=",")
data = data.drop(columns = ['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'])
date_func = lambda x: datetime.datetime.strptime(x, '%m/%d/%Y') 
data.rename(columns={'Company Symbol':"Company_Symbol", 'Trancript Name':"Transcript_Name", 'Extracted Month':"Exracted_Month",
       'Extracted Year':"Extracted_year", 'Extracted Date':"Extracted_Date", 'Create Date':"Created_Date"}, inplace=True)

data["create_date2"] = data["Created_Date"].map(date_func)

## Manipulate Trancript data to know the previous instance of transcript release

data["next_publish_date"] =  data.sort_values(by='create_date2').groupby(['Company_Symbol'])['create_date2'].shift(-1)
data["prev_publish_date1"] =  data.sort_values(by='create_date2').groupby(['Company_Symbol'])['create_date2'].shift(1)
data["prev_publish_date2"] =  data.sort_values(by='create_date2').groupby(['Company_Symbol'])['create_date2'].shift(2)
data["prev_publish_date3"] =  data.sort_values(by='create_date2').groupby(['Company_Symbol'])['create_date2'].shift(3)
data["prev_publish_date4"] =  data.sort_values(by='create_date2').groupby(['Company_Symbol'])['create_date2'].shift(4)

data["def_date"] = datetime.datetime.strptime('2099-01-01', '%Y-%M-%d')

## Impute next publish date with def date in case of nulls 

data["next_publish_date"] = data.next_publish_date.combine_first(data.def_date)

In [74]:
data.columns

Index(['Company_Symbol', 'Transcript_Name', 'Date', 'Exracted_Month',
       'Extracted_year', 'Extracted_Date', 'Created_Date', 'create_date2',
       'next_publish_date', 'prev_publish_date1', 'prev_publish_date2',
       'prev_publish_date3', 'prev_publish_date4', 'def_date'],
      dtype='object')

In [75]:
data.head()
data.count()

Company_Symbol        34534
Transcript_Name       34534
Date                  34534
Exracted_Month        34534
Extracted_year        34534
Extracted_Date        34534
Created_Date          34534
create_date2          34534
next_publish_date     34534
prev_publish_date1    32807
prev_publish_date2    31158
prev_publish_date3    29568
prev_publish_date4    28021
def_date              34534
dtype: int64

In [76]:
## Read stanford CALS data 
cals_data = pd.read_csv('D:\\Chubb\\Data\\Stanford CALS Database 0727.csv')

## Double check the filing dates

cals_data["filing_date_2"] = cals_data["Filing Date"].apply(pd.to_datetime)

## Create flag for CALS

cals_data["is_cals"] = 1

In [77]:
cals_data.head()
cals_data.count()

Filing Name       5007
Filing Date       5007
District Court    5007
Exchange          5007
Ticker            5007
filing_date_2     5007
is_cals           5007
dtype: int64

In [78]:
## Pandas SQL to implement Left Join like behavior with inequality join

sql_query = '''Select a.*, b.filing_date_2, coalesce(b.is_cals,0) as is_cals FROM data a left join 
cals_data b ON a.Company_Symbol = b.Ticker 
AND a.create_date2 < b.filing_date_2 AND a.next_publish_date >= b.filing_date_2'''

data2 = ps.sqldf(sql_query)

In [80]:
data2.to_csv('D:\\Chubb\\Company check\\transcript_link_with_cals.csv', sep=",")

In [27]:
## Write function to get count of CALS with previous 4 quarters information avauilable as well

i = 0

## iterate over dataset
series_tickers = set(data["Company Symbol"])
for tickers in series_tickers:
    # Filter data for tikcer 
    subset_data = data[data["Company Symbol"] == ticker]
    # Iterate over rows of filtered data 
    
    

In [29]:
len(series_tickers)

1727

### Exploring data for Sanity checks

In [155]:
## See the records where next_publish date and current_publish date is zero 

data[data["day_diff"] >= 60].describe()

## See the records where next_publish date has been marked as default value 

data[data["next_publish_date"] == datetime.datetime.strptime('2099-01-01', '%Y-%M-%d')].count()

## Verify the summary for CALs data

cals_data["filing_date_2"].describe()

count                    5007
unique                   3114
top       2001-12-06 00:00:00
freq                       19
first     1996-01-19 00:00:00
last      2018-07-20 00:00:00
Name: filing_date_2, dtype: object

In [220]:
data2.describe()

Scraped date      day_diff  cal_def_value       is_cals
count  2.133800e+04  21455.000000        21455.0  21455.000000
mean   2.018071e+07    743.149429            0.0      0.007691
std    2.874252e+00   4361.946681            0.0      0.087360
min    2.018071e+07      0.000000            0.0      0.000000
25%    2.018071e+07     54.000000            0.0      0.000000
50%    2.018071e+07     90.000000            0.0      0.000000
75%    2.018072e+07     94.000000            0.0      0.000000
max    2.018072e+07  30463.000000            0.0      1.000000

## Ignore this part, We can research more on this later

In [197]:
## Merge Trancript and CALs data to get a combined AD with FLAG

data = data.merge(cals_data, how='left', left_on=["input company name"], right_on= ["Ticker"])

#data["filing_date_2"] = data.filing_date_2.combine_first(data.next_publish_date)

#np.where(data["filing_date_2"].is_null() == TRUE, data.next_publish_date, data["filing_date_2"])
data["is_cals"] = data.is_cals.combine_first(data.cal_def_value)



In [202]:
## Exporting data to csv 
data_x = data[(data.published_date_time < data.filing_date_2) & (data.filing_date_2 <= data.next_publish_date)]
data_y = data[(data.published_date_time < data.filing_date_2) & (data.filing_date_2 > data.next_publish_date)]
data_z = data[(data.published_date_time > data.filing_date_2) & (data.filing_date_2 < data.next_publish_date)]
#data.to_csv('D:\\Chubb\\Data\\transcript_with_cals.csv')